In [1]:
import pandas as pd
import re

import numpy as np
import pandas as pd
from pprint import pprint
from collections import Counter

#nltk stopwords
from nltk.corpus import stopwords


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

#TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

### Read Posts

In [2]:
name = "LatinosForBiden"
terms = "biden|democrats|democrat"
candidate = pd.read_csv(name+".csv")
candidate.shape

(4971, 31)

In [3]:
candidate.columns

Index(['Group Name', 'User Name', 'Facebook Id', 'Likes at Posting', 'Created',
       'Type', 'Likes', 'Comments', 'Shares', 'Love', 'Wow', 'Haha', 'Sad',
       'Angry', 'Care', 'Video Share Status', 'Post Views', 'Total Views',
       'Total Views For All Crossposts', 'Video Length', 'URL', 'Message',
       'Link', 'Final Link', 'Image Text', 'Link Text', 'Description',
       'Sponsor Id', 'Sponsor Name', 'Total Interactions',
       'Total Interactions (weighted  —  Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x )'],
      dtype='object')

## Filter the most relevant columns

In [4]:
content = candidate[['Message','Description','Link','Group Name','Likes', 'Comments', 'Shares', 'Love', 'Wow', 'Haha', 'Sad',
       'Angry', 'Care','Total Interactions']].copy()
content.shape

(4971, 14)

### Add column to concat Message and Description

In [5]:
concat_message = content['Message'].fillna('') + (' ' + content['Description']).fillna('')
content['MessageDescr'] = concat_message

### Just keep those groups that have Trump or Republican in their name

In [6]:
groups_candidate = content['Group Name'].str.lower().reset_index()
final_groups = groups_candidate[groups_candidate['Group Name'].str.contains(terms)].copy()
final_groups = final_groups['Group Name'].unique()
final_groups = pd.DataFrame(final_groups)
final_groups.columns = ['Group Name']
final_groups.to_csv(name+"_groups.csv")
final_groups.shape

(191, 1)

In [7]:
content['Group Name'] = content['Group Name'].str.lower()
content.shape

(4971, 15)

### Keep the posts from the preselected groups

In [8]:
content = content[content['Group Name'].isin(final_groups['Group Name'])].copy()
content.shape

(640, 15)

## Tokenize words and Clean-up text

In [9]:
content['cleanMessage'] = content['MessageDescr'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
content['cleanMessage'].shape

(640,)

In [10]:
# Convert to list
data = content.cleanMessage.tolist()

In [11]:
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['ricky', 'martin', 'one', 'of', 'the', 'biggest', 'hispanic', 'pop', 'icon', 'openly', 'supporting', 'joe', 'biden', 'pleads', 'for', 'all', 'his', 'followers', 'to', 'support', 'joe', 'biden', 'for', 'presdident', 'el', 'artista', 'que', 'ha', 'expresado', 'en', 'mas', 'de', 'una', 'ocasion', 'su', 'contrariedad', 'la', 'politica', 'de', 'trump', 'participa', 'en', 'un', 'encuentro', 'del', 'candidato', 'democrata', 'con', 'los', 'latinos', 'luis', 'fonsi', 'eva', 'longoria', 'tambien', 'pidieron', 'su', 'voto', 'en', 'dicho', 'evento']]


#### Creating Bigram and Trigram Models

In [13]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['ricky_martin', 'one', 'of', 'the', 'biggest', 'hispanic', 'pop', 'icon', 'openly', 'supporting', 'joe', 'biden', 'pleads', 'for', 'all', 'his', 'followers', 'to', 'support', 'joe', 'biden', 'for', 'presdident', 'el', 'artista', 'que', 'ha', 'expresado', 'en', 'mas', 'de', 'una', 'ocasion', 'su', 'contrariedad', 'la', 'politica', 'de', 'trump', 'participa', 'en', 'un', 'encuentro', 'del', 'candidato', 'democrata', 'con', 'los', 'latinos', 'luis', 'fonsi', 'eva_longoria', 'tambien', 'pidieron', 'su', 'voto', 'en', 'dicho', 'evento']


In [14]:
stop_words = stopwords.words('english')
stop_words.extend(stopwords.words('spanish'))
stop_words.extend(['su','sus','al','de','en','el'])
stop_words.extend(['like','would','get','many','much'])

## Remove Stopwords, Make Bigrams and Lemmatize

In [15]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts,stop_words_lang):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words_lang] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

## Bag of Words

In [16]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words,stop_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#!python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

[['ricky_martin', 'big', 'hispanic', 'pop', 'icon', 'openly', 'support', 'plead', 'follower', 'support', 'ocasion'], ['call', 'activism', 'need', 'sanity']]


In [17]:
def combine_words(word1,word2,dictionary):

    if word1 in dictionary and word2 in dictionary:
        concat_name = word1+" / "+word2
        dictionary[concat_name] = dictionary[word1] + dictionary[word2]
        dictionary.pop(word1, None)
        dictionary.pop(word2, None)
        print(concat_name, dictionary[concat_name])
    return dictionary

In [18]:
flat_list_words = [item for sublist in data_lemmatized for item in sublist]
count_words = Counter(flat_list_words)

In [19]:
final_dict = combine_words('donald','trump',count_words)
final_dict = combine_words('joe','biden',count_words)
final_dict = combine_words('kamala','harris',count_words)
final_dict = combine_words('american','america',count_words)
final_dict = combine_words('estados','unidos',count_words)
final_dict = combine_words('voters','vote',count_words)
final_dict = combine_words('mexico','mexican',count_words)
final_dict = combine_words('casa','blanca',count_words)
final_dict = combine_words('venezuela','venezuelans',count_words)
final_dict = combine_words('latino','latinos',count_words)
final_dict = combine_words('american / america','americans',count_words)
final_dict = combine_words('voters / vote','voting',count_words)
final_dict = combine_words('puerto','rico',count_words)
final_dict = combine_words('communism','communist',count_words)
final_dict = combine_words('ee','uu',count_words)
final_dict = combine_words('united','states',count_words)

joe / biden 429
latino / latinos 475
communism / communist 28


In [20]:
final_dict.most_common(1000)

[('trump', 824),
 ('vote', 475),
 ('latino / latinos', 475),
 ('joe / biden', 429),
 ('voter', 392),
 ('state', 271),
 ('people', 253),
 ('make', 234),
 ('election', 229),
 ('say', 224),
 ('go', 187),
 ('hispanic', 173),
 ('support', 169),
 ('campaign', 157),
 ('plan', 151),
 ('know', 147),
 ('democratic', 145),
 ('community', 129),
 ('help', 129),
 ('want', 125),
 ('win', 123),
 ('white', 116),
 ('take', 112),
 ('month', 107),
 ('need', 99),
 ('see', 98),
 ('live', 98),
 ('show', 97),
 ('time', 94),
 ('life', 94),
 ('share', 94),
 ('come', 94),
 ('downplay', 94),
 ('also', 93),
 ('group', 90),
 ('country', 88),
 ('lose', 88),
 ('year', 87),
 ('call', 85),
 ('former', 85),
 ('black', 83),
 ('debate', 83),
 ('care', 83),
 ('poll', 82),
 ('tell', 82),
 ('american', 80),
 ('virus', 80),
 ('accord', 79),
 ('well', 76),
 ('spend', 73),
 ('event', 73),
 ('family', 72),
 ('president', 72),
 ('point', 72),
 ('question', 71),
 ('first', 70),
 ('talk', 70),
 ('pandemic', 70),
 ('key', 69),
 ('ri

## TD-IDF

In [21]:
docs = ([' '.join(ele) for ele in data_words_nostops] )

In [22]:
cv=CountVectorizer() 
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(640, 6541)

In [23]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [24]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

,idf_weights
biden,1.232548
latinos,1.246423
trump,1.739444
joe,1.786276
vote,2.092391
...,...
incomes,6.769882
inconvenient,6.769882
incorporated,6.769882
improve,6.769882


In [25]:
# count matrix 
count_vector=cv.transform(docs) 
 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

In [26]:
feature_names = cv.get_feature_names() 
 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[0] 
 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df = df[(df != 0).all(1)]
df.sort_values(by=["tfidf"],ascending=False).to_csv(name+"_tfidf.csv")

In [27]:
df

,tfidf
artista,0.196285
biden,0.076026
biggest,0.141025
candidato,0.170153
contrariedad,0.196285
democrata,0.196285
dicho,0.196285
encuentro,0.196285
eva,0.170153
evento,0.174908


In [28]:
word = df.index[0]
content["lowercase"] = content['MessageDescr'].apply(lambda x: x.lower())
word_messages = content[content['lowercase'].str.contains(word)]

word_messages = word_messages['lowercase'].value_counts(ascending=False).rename_axis('unique_messages').reset_index(name='counts')
word_messages.to_csv(word +"_"+ name+"_messages.csv")
word_messages

,unique_messages,counts
0,ricky martin one of the biggest hispanic pop i...,2
1,for all latinos in miami who support biden... ...,1
2,https://www.youtube.com/watch?v=i2ynxayspew es...,1


### Repeated messages

In [29]:
content['Total Interactions']=content['Total Interactions'].astype(str).str.replace(',', '').astype(int)
results = content.groupby(['MessageDescr']).aggregate({'Group Name':['count',' |'.join],'MessageDescr': 'count','Likes': 'sum','Comments': 'sum', 'Shares': 'sum', 'Love': 'sum', 'Wow': 'sum', 'Haha': 'sum', 'Sad': 'sum',
       'Angry': 'sum', 'Care': 'sum','Total Interactions':'sum'})
results.rename(columns={"MessageDescr": "Counts",'Group Name':"Num.Groups"}, inplace=True)


results.to_csv("mostpopularLatinosFor"+name+".csv")

In [30]:
#results.sort_values(by='Counts',ascending=False)
results

Num.Groups  \
                                                        count   
MessageDescr                                                    
                                                            3   
 "Joe Biden is a puppet of the extreme left who...          1   
 "Voter turnout among Latinos in Florida could ...          1   
 #TodosConBiden                                             1   
 (CNN)Former President Barack Obama delivered a...          1   
...                                                       ...   
👏🏻👏🏻👏🏻💙💙💙LATINOS FOR BIDEN 🇺🇸🇺🇸👊🏻👊🏻🇺🇸🇵🇷🇨🇺🇨🇴🇪🇨🇦🇷...          1   
💙 Official Webstore of Biden for President                  1   
📍Miami-Dade, FL #TodosConBidenCaravan 📍Miami-Da...          6   
😂😂😂😂😂 Kamala Harris made a surprise visit to Mi...          1   
😂😆 Why do some people vote against their own in...          1   

                                                                                                       \
                                                                                                 join   
MessageDescr                                                                                            
                                                    social democrats usa-socialist party, usa |rid...   
 "Joe Biden is a puppet of the extreme left who...                                  ridin' with biden   
 "Voter turnout among Latinos in Florida could ...                                  cubanos con biden   
 #TodosConBiden                                                                     cubanos con biden   
 (CNN)Former President Barack Obama delivered a...  why i love joe biden-kamala harris / why i hat...   
...                                                                                               ...   
👏🏻👏🏻👏🏻💙💙💙LATINOS FOR BIDEN 🇺🇸🇺🇸👊🏻👊🏻🇺🇸🇵🇷🇨🇺🇨🇴🇪🇨🇦🇷...                                  cubanos con biden   
💙 Official Webstore of Biden for President                                     joe biden for colorado   
📍Miami-Dade, FL #TodosConBidenCaravan 📍Miami-Da...  biden for florida |joe biden for florida |cuba...   
😂😂😂😂😂 Kamala Harris made a surprise visit to Mi...  americans against democratic and republican party   
😂😆 Why do some people vote against their own in...                                  biden/harris 2020   

                                                   Counts Likes Comments  \
                                                    count   sum      sum   
MessageDescr                                                               
                                                        3    10       22   
 "Joe Biden is a puppet of the extreme left who...      1     0        0   
 "Voter turnout among Latinos in Florida could ...      1     9        0   
 #TodosConBiden                                         1    11        0   
 (CNN)Former President Barack Obama delivered a...      1    11        1   
...                                                   ...   ...      ...   
👏🏻👏🏻👏🏻💙💙💙LATINOS FOR BIDEN 🇺🇸🇺🇸👊🏻👊🏻🇺🇸🇵🇷🇨🇺🇨🇴🇪🇨🇦🇷...      1    15        0   
💙 Official Webstore of Biden for President              1    16        0   
📍Miami-Dade, FL #TodosConBidenCaravan 📍Miami-Da...      6   332       51   
😂😂😂😂😂 Kamala Harris made a surprise visit to Mi...      1     1        0   
😂😆 Why do some people vote against their own in...      1     9        7   

                                                   Shares Love Wow Haha Sad  \
                                                      sum  sum sum  sum sum   
MessageDescr                                                                  
                                                        4    3   0    0   0   
 "Joe Biden is a puppet of the extreme left who...      0    0   0    0   0   
 "Voter turnout among Latinos in Florida could ...      1    2   0    0   0   
 #TodosConBiden                                         1    2   0    0   0   
 (CNN)Former President Barack Obama delivered a... 